In [1]:
import shutil
import arcpy
from archaic import FeatureClass

geodatabase = ".data/test.geodatabase"
shutil.copyfile(".data/world.geodatabase", geodatabase)
arcpy.env.workspace = geodatabase  # type: ignore

In [2]:
class City:
    objectid: int
    city_name: str
    pop: int
    shape: arcpy.PointGeometry

feature_class = FeatureClass[City]("cities")

for city in feature_class.read("CITY_NAME LIKE 'X%'"):
    print(city.city_name, city.shape.WKT)

Xuddur POINT (43.866702400295594 4.1832989176110118)
Xam Nua POINT (104.03330444404492 20.41669795414731)
Xiangkhoang POINT (103.3667013487883 19.333295965436491)
Xining POINT (101.7649984164438 36.645003073233283)
Xi'an POINT (108.88335942879468 34.265700039625187)
Xai-Xai POINT (33.640002411582088 -25.040996950149065)


In [3]:
class City:
    objectid: int
    city_name: str
    pop: int
    shape: arcpy.PointGeometry

feature_class = FeatureClass[City]("cities")

for city in feature_class.read(wkid=3857, sql_clause=(None, "ORDER BY POP DESC LIMIT 5")):
    print(city.city_name, city.pop, city.shape.WKT)

Shanghai 26121000 POINT (13522312.551844737 3664998.7120913547)
Mexico City 21942666 POINT (-11034831.133193085 2205279.1188980993)
Beijing 20781000 POINT (12956256.9482968 4852319.2771486267)
Lagos 15825000 POINT (367354.68730720528 719532.42122153542)
Buenos Aires 15416728 POINT (-6506605.7240423262 -4112697.4659454711)


In [4]:
class City:
    objectid: int
    globalid: str
    city_name: str
    pop: int
    shape: arcpy.PointGeometry

feature_class = FeatureClass[City]("cities")

city = feature_class.get(7)

if city:
    print(city.city_name, city.pop, city.shape.JSON)

Puerto Presidente Stroessner 0 {"x":-54.622997604156069,"y":-25.526996972632162,"spatialReference":{"wkid":4326,"latestWkid":4326}}


In [5]:
from dataclasses import dataclass

@dataclass
class City:
    objectid: int
    globalid: str
    city_name: str
    pop: int
    shape: arcpy.PointGeometry

feature_class = FeatureClass[City]("cities")

print(feature_class.get(7))
print(feature_class.get("{8BB6B36D-74E7-4B88-AD7F-BBA319CEF7A7}"))

City(objectid=7, globalid='{8BB6B36D-74E7-4B88-AD7F-BBA319CEF7A7}', city_name='Puerto Presidente Stroessner', pop=0, shape=<PointGeometry object at 0x1e3ce0c3940[0x1e3ce0c33f0]>)
City(objectid=7, globalid='{8BB6B36D-74E7-4B88-AD7F-BBA319CEF7A7}', city_name='Puerto Presidente Stroessner', pop=0, shape=<PointGeometry object at 0x1e3ce0c3400[0x1e3ce0c3930]>)


In [6]:
from dataclasses import dataclass
from archaic import GlobalID, ObjectID

@dataclass
class City(ObjectID, GlobalID):
    city_name: str
    pop: int
    shape: arcpy.PointGeometry

feature_class = FeatureClass[City]("cities")

feature_class.get(7)

City(globalid='{8BB6B36D-74E7-4B88-AD7F-BBA319CEF7A7}', objectid=7, city_name='Puerto Presidente Stroessner', pop=0, shape=<PointGeometry object at 0x1e3ce0c3490[0x1e3ce0c3510]>)

In [7]:
from dataclasses import dataclass
from archaic import GlobalID, ObjectID

@dataclass
class City(ObjectID, GlobalID):
    city_name: str
    pop: int
    shape: arcpy.PointGeometry

feature_class = FeatureClass[City]("cities")

city = City("Lillooet", 1234, arcpy.PointGeometry(arcpy.Point(-120, 50)))

feature_class.insert(city)

City(globalid='{E4D7F15F-FA48-4D94-8388-BFF6FD14E440}', objectid=3741, city_name='Lillooet', pop=1234, shape=<PointGeometry object at 0x1e3ce0d21f0[0x1e3ce0d2f00]>)

In [8]:
from dataclasses import dataclass
from typing import Any, Union
from archaic import GlobalID, ObjectID

@dataclass
class City(ObjectID, GlobalID):
    city_name: str
    pop: int
    shape: Union[arcpy.PointGeometry, Any]

feature_class = FeatureClass[City]("cities")

city = City("Lillooet", 1234, (-120, 50))

inserted_city = feature_class.insert(city)
inserted_city.shape.buffer(1).WKT  # type: ignore

'MULTIPOLYGON (((-120.0000000001998 51.000000000000057, -120.03109986246965 50.999516282292042, -120.06216963763129 50.998065597133653, -120.09317926768388 50.995649347969078, -120.12409875281307 50.99226987236333, -120.15489818041389 50.987930439740815, -120.1855477540293 50.982635248222323, -120.21601782217628 50.976389420563663, -120.24627890703181 50.969198999199726, -120.27630173295063 50.961070940398784, -120.30605725478769 50.952013107532785, -120.33551668599733 50.942034263470049, -120.36465152648246 50.931144062097715, -120.39343359016655 50.919353038982294, -120.42183503226187 50.906672601177128, -120.44982837620744 50.893115016186847, -120.47738654025093 50.878693400099323, -120.50448286364845 50.863421704896723, -120.53109113245708 50.847314704957832, -120.55718560489524 50.830387982764854, -120.58274103624611 50.812657913828311, -120.60773270328033 50.794141650844807, -120.63213642817414 50.774857107102946, -120.65592860189975 50.754822939153314, -120.67908620706569 50.734

In [9]:
from dataclasses import dataclass
from typing import Any, Union
from archaic import EditTracking, GlobalID, ObjectID

@dataclass()
class City(ObjectID, GlobalID, EditTracking):
    city_name: str
    pop: int
    shape: Union[arcpy.PointGeometry, Any]

feature_class = FeatureClass[City]("cities")

city = City("Lillooet", 1234, (-120, 50))

inserted_city = feature_class.insert(city)

vars(inserted_city)

{'city_name': 'Lillooet',
 'pop': 1234,
 'shape': <PointGeometry object at 0x1e3c4c91130[0x1e3ce0d21b0]>,
 'created_user': 'JIRO',
 'created_date': datetime.datetime(2024, 5, 1, 14, 23, 35),
 'last_edited_user': 'JIRO',
 'last_edited_date': datetime.datetime(2024, 5, 1, 14, 23, 35),
 'globalid': '{A4F95FD4-0CEF-4C49-B3C9-EEE1697C8AD9}',
 'objectid': 3743}